# Developing an Information Retrieval System with Advanced Boolean Search

## AmirAli Amini - 610399102

#### HW2



# توضیحات مسئله و چالش ها و بهبود ها

بزرگ ترین چالش مسئله ساختار پستینگ لیست که برای سیو کردن مقادیر استفاده میشود بود چرا که باید به صورتی ذخیره شوند که برای هر داکیومنت علاوه بر شامل بود کلمه ، ایندکس های آن کلمه هم ذخیره شود.

در این مسئله از ساختاری سریع تر از لینک لیست برای دسترسی به دیتا استفاده کردم

همچنین از باینری سرچ برای بهبود سرعت پیدا کردن دیتا استفاده میکنم

میتوان تمام دیکشنری های استفاده شده را با ارایه دو بعدی جایگذین کرد که تاثیری در سرعت کد زده شده ندارد و تنها خوانایی کد پایین اورده میشود



## کتابخانه ها 

###  from nltk import word_tokenize :
از این کتابخانه برای تکنایز کردن داده ها به این دلیل که توکنایز کردن دیتا سریع تر میشه استفاده کردم

###  from nltk.corpus import stopwords :
از این کتابخانه برای دریافت استاپینگ ورد های زبان انگلیسی استفاده کردم

###  import string:
از این کتابخانه برای دریافت پانچویشن های زبان انگلیسی استفاده کردم

###  import numpy as np:
از این کتابخانه برای جمع یک عدد با تمام اعضای یک آرایه استفاده کردم

###  import copy:
از این کتابخانه برای دیپ کپی کردن ارایه استفاده کردم

In [760]:
from nltk.tokenize import word_tokenize


from nltk.corpus import stopwords # a library to tokenize input texts

import nltk


nltk.download('punkt')
nltk.download('stopwords') # stopping word in English language

import string # using to remove punctuation

import numpy as np

import copy

[nltk_data] Downloading package punkt to /Users/amirali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amirali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Code 
برای رسیدن به هدف سوال، یک 
"posring list"
با ساختار
list of {word : nameOfWord , docs :[list of {doc:nameOfDocument , indexes: indexes of the word if this document}]}
میسازم که هر خانه متناظر با یک کلمه است و ساختاری به صورت گفته شده دارد

در این ساختار داکیومنت هایی که دارای این کلمه هستند در این قسمت ذخیره میشووند به این صورت که ایندکس هایی که برابر آن کلمه در آن داکیومنت هستند نگهداری میشود

دلیل نگه داری اندکس کلمات در داکیومنت ها این است که در آینده بتوان با متد 

near

اختلاف مکانی دو کلمه را پیدا کرد


کلیت ساخت پستینگ لیست به این صورت است که کلمات هر داکیومنت را نگاه میکند در پستینگ لیست پس از پیدا کردن کلمه مورد نظر، مقدار ایندکس آن در داکیومنت حاضر و شماره داکیومنت ثبت میشود

توضیحات جزئی تر در کد به صورت کامنت نوشته شده است


### input:
تابع اینپوت ادرس داکیومنت ها را ورودی میگیرد و برای هر داکیومنت عملیات اضافه کردن به پستینگ لیست را انجام میدهد

### findWord:
این تابع ایندکس متناظر با کلمه سرج شده را خروجی میدهد

### find:
این در مرحله اول کویری داده شده را تجزیه میکند به ۳ کتگوری اصلی تقسیم بندی میشود

- تک کلمه
- بولین
- همسایگی

#### single word:
برای تک کلمه به صورت مستثیم از تابع فایند ورد استفاده میکنم و ایندکس داکیومنت های آن را برمیگردانم

#### boolean:
برای بولین ها داکیومنت های هر کلمه را به همانند تک کلمه پیدا میکنم  

##### AND:
اشتراک لیست های بدست آمده را برمیگردانم

##### OR:
اجتماع لیست های بدست آمده را برمیگردانم


### NEAR:
برای کلمه اول و دوم برای هرکدام یک دیکشنری از داکیومنت های دارای آن کلمه میسازم که کلید آن اندکس داکیومنت و مقدار آن ایندکس کلمه در آن داکیومنت است

برای هر دوداکیومنت مشترک در در دو کلمه ماتریس قدرمطلق فاصله متناظر با عضو های هردو را میسازم و مینیمم فاصله را بدست می آورم

اگر کوچک تر از عدد خواسته شده بود آن داکیومنت را به جواب اضافه میکنم

# input 1 , text2.txt : 
aa aa  aa bb bb bb zz zz ll mm mm  aa aa ll 


amir ali amini amir hossein hasani


# input 2 , test.tst:
amirali , amini, hastam salam. amirali , amirali aa bb

In [761]:
class node :
    def __init__(self,isWord=False, myChar=None , myWord=None):
        self.myChar = myChar
        self.myWord = myWord
        self.nodes = {}
        self.seenChars = []
        self.isAWrod = isWord
    
    def addNode(self ,char , isWord = False):
        if (char not in self.seenChars):
            newNode = node(myChar=char , myWord=self.myWord+char if self.myWord else char , isWord=isWord)
            self.nodes[char] = newNode
            self.seenChars.append(char)
            return newNode
        if isWord:
            self.nodes[char].makeWord()
        return self.nodes[char]
        
    def makeWord(self):
        self.isAWrod=True

    def getWord (self):
        return self.myWord
    
    def getIsWord (self):
        return self.isAWrod

    def getChar (self):
        return self.myChar
    
    def includesChar(self, char):
        return char in self.seenChars
    
    def getSeenNodes (self):
        return [self.nodes[char] for char in self.seenChars]




In [762]:



class tryTree :
    def __init__ (self):
        self.root = node()

    def insertWord(self, word):
        currentNode  = self.root
        for char in range(len(word)): 
            currentNode = currentNode.addNode(word[char] , isWord = char == len(word)-1)


    def insertWordPermutation (self,inputWord):
        word = inputWord+"$"
        for index in range(len(word)):
            self.insertWord(word[index:]+word[:index])

    def BFS(self , node):
        result = [node]
        for child in node.getSeenNodes():
            result+= self.BFS(child)
        return result
    
    def getBFSWords (self , node):
        ls = self.BFS(node)
        return [node.getWord() for node in ls if node.getIsWord()]
    
    def prefixBFS (self, prefix= ''):
        currentNode = self.root
        for char in range(len(prefix)): 
            currentNode = currentNode.addNode(prefix[char])
        ls = self.getBFSWords(currentNode)
        return ls

    
    def find(self, inputWord):
        word = inputWord
        if ('*' not in word):
            return []
        # if inputWord[-1] != "*":
        word +='$'
        startCount = word.count('*')
        if (startCount ==1):
            index = word.find('*')
            prefix = word[index+1:]+word[:index]
            ls = self.prefixBFS(prefix=prefix)
            st = set()
            for w in ls :
                index = w.find('$')
                st.add(w[index+1:]+w[:index])
            return list(st)
        elif(startCount==2):
            while (word[-1]!='*'):
                word = word[-1:]+word[:-1]
            splitWord = word[:-1].split('*')
            if len(splitWord[0])< len(splitWord[1]):
                splitWord[0], splitWord[1] = splitWord[1],splitWord[0]
            
            ls = [i for i in self.prefixBFS(prefix=splitWord[0]) if splitWord[1] in i]

            st = set()
            for word in ls :
                index = word.find('$')
                st.add(word[index+1:]+word[:index])
            return list(st)




    def prnt (self,node=None):
        root = self.root
        if (node):
            root = node
        print (self.getBFSWords(root) )
    


In [763]:

class searchEngine:
    def __init__(self , debug = False) -> None: # constructor of class
        self.debug = debug
        self.postingList =[]
        self.files=[] 
        self.stop = set(stopwords.words('english') + list(string.punctuation)) # all extra expression which should ignore
        self.tryTree = tryTree()

        # structure of postingList : list of {word : nameOfWord , docs :[list of {doc:nameOfDocument , indexes: indexes of the word if this document}]}


    # binary search to find a word in posting list
    def searchPostingList(self, word):
        s= 0 
        e = len(self.postingList)
        if e <=0 :
            return 0
        e-=1
        while (1):
            if (e-s < 2):
                if (self.postingList[e]["word"] < word):
                    return e+1
                if (self.postingList[e]["word"] == word):
                    return e
                if (self.postingList[s]["word"] >= word):
                    return s

                return e
            mid = (s+e)/2
            mid = int(mid)
            if (word<self.postingList[mid]["word"]):
                e=mid
            elif (word> self.postingList[mid]["word"]):
                s = mid
            else :
                return mid
            
        # {word:str, indexes:list(int)}
    # binary search to find a word in each dictionary
    def searchDictionary(self, word,ls):
        s= 0 
        e = len(ls)
        if e <=0 :
            return 0
        e-=1
        while (1):
            if (e-s < 2):
                if (ls[e]["doc"] < word):
                    return e+1
                if (ls[e]["doc"] == word):
                    return e
                if (ls[s]["doc"] >= word):
                    return s

                return e
            mid = (s+e)/2
            mid = int(mid)
            if (word<ls[mid]["doc"]):
                e=mid
            elif (word> ls[mid]["doc"]):
                s = mid
            else :
                return mid


            
        # {doc:number, indexes:list(int)}



    def addToPostingList(self, tokenizedText: list[str],docIndex:int): # add tokenized word in posting list 
        for i in range(len(tokenizedText)):
            word = tokenizedText[i]
            index = self.searchPostingList(word) # find index of word in posting list
            if (len(self.postingList)>index): # check if index is not larger than posting list (if word is bigger that all words, search function returns len(postingList)+1)
                if (self.postingList[index]["word"] == word): # check if index is the index of the word
                    if (self.postingList[index]["docs"][-1]["doc"] == docIndex): # if we have already added the document index 
                        self.postingList[index]["docs"][-1]["indexes"].append(i) # as we read tokens in order of their index, we need to add token in end of the list

                    else:
                        self.postingList[index]["docs"].append({"doc":docIndex,"indexes":[i] }) # if we have not already added the document and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list 

                else :
                    self.postingList[index:index]= [({"word":word , "docs":[{"doc":docIndex , "indexes":[i]}]})] # word is bigger that all other words => we can append it to end of the list

            else :
                self.postingList.append({"word":word , "docs":[{"doc":docIndex , "indexes":[i]}]}) # we have not already added the word and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list

            self.tryTree.insertWordPermutation(word)


    def input (self, filePath: list[str]): # input paths of inputs
        for i in range(len(filePath)): # for files in input
            file = open(filePath[i],'r',encoding='cp1252') # open the file
            text = file.read() # read the file
            file.close()  # close the file
            # tokenize text and ignore stopping words using nltk library 
            tokenizedText = [word for word in word_tokenize(text.lower(),preserve_line=False) if word not in self.stop] 
            print (f'document {i+1} : {filePath[i]}')
            # print(tokenizedText)
            self.addToPostingList(tokenizedText , i+1) # i indicates to index of document we are reading
        


    def findWord(self, word): # this function use our binary search function to find word in posting list and if the word is not included in the list, returns -1
        index = self.searchPostingList(word)
        if (index< len(self.postingList)):
            if (self.postingList[index]["word"] == word):
                return index # real index of the word
        return -1 # word is not in the posting list


    def find(self , query:str): # split the query and find the result 
        splitQuery = query.lower().split()
        if (len(splitQuery)==1): # query is only one word
            index = self.findWord(splitQuery[0])
            if (index>-1):
                return [ i["doc"] for i in self.postingList[index]["docs"]]
            return []

        else: 
            index1 = self.findWord(splitQuery[0])
            index2 = self.findWord(splitQuery[2])
            if splitQuery[1] in ["and" ,"or", "AND", "OR"]: # boolean condition
                
                if (splitQuery[1] in ["and","AND"]): # and condition
                    if(index1!=-1 and index2!=-1): # check if there are result for both of words
                        docs1 =  set([ i["doc"] for i in self.postingList[index1]["docs"]]) # find document of word one 
                        docs2 =  set([ i["doc"] for i in self.postingList[index2]["docs"]]) # find document of word two 
                        # print(self.postingList[index1]["docs"] , self.postingList[index2]["docs"])
                        return list(docs1.intersection(docs2)) # make intersection of two results
                    return []
                    
                if (splitQuery[1] in ["or","OR"]): # or condition
                    if(index1!=-1 and index2!=-1): # check if there are result for both of words
                        docs1 =  set([ i["doc"] for i in self.postingList[index1]["docs"]])
                        docs2 =  set([ i["doc"] for i in self.postingList[index2]["docs"]])
                        # print(self.postingList[index1]["docs"] , self.postingList[index2]["docs"])
                        return list(docs1.union(docs2))
                    if(index1!=-1): return [ i["doc"] for i in self.postingList[index1]["docs"]] # check if there is result for first word
                    return [ i["doc"] for i in self.postingList[index2]["docs"]] # check if there is result for second word

            else : # near condition
                nearNumber = int(splitQuery[1].split('/')[1]) # find near 
                if(index1!=-1 and index2!=-1): # check if there are result for both of words
                    docs1 =   {i["doc"]:i["indexes"] for i in self.postingList[index1]["docs"]} # find document of word one and make dictionary for result
                    docs2 =   {i["doc"]:i["indexes"] for i in self.postingList[index2]["docs"]} # find document of word two and make dictionary for result

                    result = [] 
                    keysOfDocs2 = docs2.keys()
                    for key, value in docs1.items():
                        if (key in keysOfDocs2):
                            d1 = np.array(value) # indexes of word in first document
                            d2 = np.array(copy.deepcopy(docs2[key])) # indexes of word in second document
                            distanceMatrix = np.array([[abs(i - j )for i in d1] for j in d2]) # distance matrix
                            if (distanceMatrix.min() <= nearNumber): # check validation
                                result.append(key)


                            # example of distanceMatrix
                            # 
                            # [[ 0  1  2 10 11]
                            # [ 1  0  1  9 10]
                            # [ 2  1  0  8  9]
                            # [10  9  8  0  1]
                            # [11 10  9  1  0]]


                    return result
                return []
            






# HW2 : codes of homework 2

    def findDistance (self,inputWord, baseWord ,printOff=True ): # this function is used to find distance of each word if posting list with input word
        inWord = " "+inputWord
        bWord = " "+baseWord
        distanceMatrix = [[0]* len(bWord) for _ in range(len(inWord))] # quantified the matrix
        for i in range(len(bWord)):
            distanceMatrix[0][i]=i
        for i in range(len(inWord)):
            distanceMatrix[i][0]=i

        for row in range(1,len(inWord)): # fill the matrix using dynamic programming algorithm
            for column in range(1,len(bWord)):
                deleteScore = distanceMatrix[row-1][column] +1
                insertScore = distanceMatrix[row][column-1] +1
                copyOrReplaceScore = distanceMatrix[row-1][column-1] 
                if bWord[column] != inWord[row]:
                    copyOrReplaceScore+=1
                distanceMatrix[row][column] = min(deleteScore , insertScore , copyOrReplaceScore)
                self.debugPrint (f'min is {distanceMatrix[row][column]} for {(deleteScore , insertScore , copyOrReplaceScore)} {row}, {column}' ,printOff=printOff)
 
        self.debugPrint(np.array(distanceMatrix),printOff=printOff)
        return distanceMatrix[-1][-1]



    def spellCheckingSingleWord (self,inputWord,printOff=True): # call "def findDistance ()" for all words in posting list with input word
        index = self.searchPostingList(inputWord)
        if (index< len(self.postingList)):
            if (self.postingList[index]["word"] == inputWord):
                return [inputWord]
            
        allWords = [item['word'] for item in self.postingList]
        distances = [[] for _ in range(100) ]
        for baseWord in allWords:
            currentDistance = self.findDistance(inputWord, baseWord ,printOff=True)
            distances[currentDistance].append(baseWord)
        self.debugPrint (distances,isMatrix=True, printOff=printOff)
        for item in distances:
            if(len(item)):
                return item
        return []
            

    def spellCheckingExpression(self, expression, printOff = True):
        tokenizedExpression= [word for word in word_tokenize(expression.lower(),preserve_line=False) if word not in self.stop] 
        answersLists = []
        for word in tokenizedExpression:
            answersLists.append(self.spellCheckingSingleWord(word))
        self.debugPrint(answersLists, printOff=printOff)

        results = []

        for ls in answersLists:
            temp =[]
            if (len(results)):
                for wordResult in results:
                    for word in ls:
                        temp.append(f'{wordResult} {word}')
                results = temp
            else :results = copy.copy(ls)

        self.debugPrint(results, printOff=printOff)
        return results
    

    def findWildCardWord (self, word ) :
        wildcardFind = self.tryTree.find(word)
        return wildcardFind


    def findWildCard (self, query ,printQueries=True) :
        if ('*' not in query):
            return self.find(query)
        splitQuery = query.lower().split()
        if (len(splitQuery) == 3):
            if '*' not in splitQuery[0]:
                splitQuery[0],splitQuery[2] = splitQuery[2],splitQuery[0]

            wildcardFind = self.tryTree.find(splitQuery[0])
            spellCheckFind = self.spellCheckingExpression(splitQuery[2])
            self.debugPrint(wildcardFind)
            results = set()
            for spellItem in spellCheckFind:
                for wildItem in wildcardFind:
                    if printQueries: print (f'query : {wildItem} {splitQuery[1]} {spellItem}')
                    results = results.union(set(self.find(f'{wildItem} {splitQuery[1]} {spellItem}')))
                # print(f'{item} {splitQuery[1]} {splitQuery[2]}' ,set(self.find(f'{item} {splitQuery[1]} {splitQuery[2]}')) ,results)
            self.debugPrint(wildcardFind,isMatrix=False)
            self.debugPrint(splitQuery,isMatrix=True)
            return list(results)
        
        else:
            wildcardFind = self.tryTree.find(splitQuery[0])
            results = set()
            for item in wildcardFind:
                results.union(set(self.find(item)))
            return list(results)
        

        







        




                        
            

            
    def debugPrint(self, value ,isMatrix = False, printOff=False):
        if (self.debug and not printOff):
            if (isMatrix):
                for row in value:
                    print(row)
            else:
                print(value)
        


    def prnt(self):
        for i in self.postingList:
            print(i)




# Test cases:

In [764]:
test = searchEngine()

test.input(['document1.txt','document2.txt','document3.txt'])


document 1 : document1.txt
document 2 : document2.txt
document 3 : document3.txt


In [765]:
test.spellCheckingExpression('amrali ani astam man',printOff=True)

['example test test test',
 'several test test test',
 'simple test test test',
 'words test test test']

In [766]:
print(test.findWildCard( "amir or ex*m*le"))

query : example or test
[1, 2, 3]


In [767]:
print(test.findWildCard('c*s and several'))

query : contains and several
query : capabilities and several
[1]


In [768]:
print(test.findWildCard('c*s or several'))

query : contains or several
query : capabilities or several
[1, 3]


In [769]:
print(test.findWildCard('c*s* near/3 several'))

query : contains near/3 several
query : capabilities near/3 several
[1]


In [770]:
print(test.findWildCard('s*l* near/3 several'))

query : simple near/3 several
query : several near/3 several
[1]


# Docs WildCards

In [771]:
import os
directory_input = ['docs/'+path for path in os.listdir('docs') if path[-3:] == 'txt']
print(directory_input)

['docs/Jerry Decided To Buy a Gun.txt', 'docs/Rentals at the Oceanside Community.txt', 'docs/Gasoline Prices Hit Record High.txt', 'docs/Cloning Pets.txt', 'docs/Crazy Housing Prices.txt', 'docs/Man Injured at Fast Food Place.txt', 'docs/A Festival of Books.txt', 'docs/Food Fight Erupted in Prison.txt', 'docs/Better To Be Unlucky.txt', 'docs/Sara Went Shopping.txt', 'docs/Freeway Chase Ends at Newsstand.txt', 'docs/Trees Are a Threat.txt', 'docs/A Murder-Suicide.txt', 'docs/Happy and Unhappy Renters.txt', 'docs/Pulling Out Nine Tons of Trash.txt']


In [772]:
test_directory = searchEngine()
test_directory.input(directory_input)

document 1 : docs/Jerry Decided To Buy a Gun.txt
document 2 : docs/Rentals at the Oceanside Community.txt
document 3 : docs/Gasoline Prices Hit Record High.txt
document 4 : docs/Cloning Pets.txt
document 5 : docs/Crazy Housing Prices.txt
document 6 : docs/Man Injured at Fast Food Place.txt
document 7 : docs/A Festival of Books.txt
document 8 : docs/Food Fight Erupted in Prison.txt
document 9 : docs/Better To Be Unlucky.txt
document 10 : docs/Sara Went Shopping.txt
document 11 : docs/Freeway Chase Ends at Newsstand.txt
document 12 : docs/Trees Are a Threat.txt
document 13 : docs/A Murder-Suicide.txt
document 14 : docs/Happy and Unhappy Renters.txt
document 15 : docs/Pulling Out Nine Tons of Trash.txt


In [773]:
result=test_directory.findWildCard("People")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[2, 3, 7, 12, 13, 14]
docs/Rentals at the Oceanside Community.txt
docs/Gasoline Prices Hit Record High.txt
docs/A Festival of Books.txt
docs/Trees Are a Threat.txt
docs/A Murder-Suicide.txt
docs/Happy and Unhappy Renters.txt


In [774]:
result=test_directory.findWildCard("Los")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[7, 11]
docs/A Festival of Books.txt
docs/Freeway Chase Ends at Newsstand.txt


In [775]:
result = test_directory.findWildCard("People and Los")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[7]
docs/A Festival of Books.txt


In [776]:
result = test_directory.findWildCard("People OR Los")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[2, 3, 7, 11, 12, 13, 14]
docs/Rentals at the Oceanside Community.txt
docs/Gasoline Prices Hit Record High.txt
docs/A Festival of Books.txt
docs/Freeway Chase Ends at Newsstand.txt
docs/Trees Are a Threat.txt
docs/A Murder-Suicide.txt
docs/Happy and Unhappy Renters.txt


In [777]:
result = test_directory.findWildCard("piano near/4 said")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[9]
docs/Better To Be Unlucky.txt


In [778]:
result = test_directory.findWildCard("pia*o near/4 said")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

query : piano near/4 said
[9]
docs/Better To Be Unlucky.txt


In [779]:
result = test_directory.findWildCard("p*a*o near/4 said")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

query : piano near/4 said
[9]
docs/Better To Be Unlucky.txt


In [780]:
result = test_directory.findWildCard("p*a near/4 said")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)


query : pizza near/4 said
query : pasadena near/4 said
[]


In [781]:
result = test_directory.findWildCard("*p*a near/4 said")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)


query : emphysema near/4 said
query : pizza near/4 said
query : pasadena near/4 said
[]


# DOCS

## Docs Spell Checking
### Input – Spell Checking

In [782]:
result = test_directory.spellCheckingExpression("festivsl funders")
for item in result:
    print (item)

festival founders


In [783]:
result = test_directory.spellCheckingExpression("contrnt")
for item in result:
    print (item)

canton
court


## Docs findWildCardWord
### Input – Information Retrieval System

In [788]:
print( test_directory.findWildCardWord("n*b*y"))


['nearby', 'nobody']


## Docs WildCardQuery
### Input – Wildcard Queries

In [785]:
result = test_directory.findWildCard("exa*le AND content")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

[]


In [793]:
result = test_directory.findWildCard("l*e AND popila")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

query : life and popular
query : love and popular
query : little and popular
query : line and popular
query : like and popular
query : lake and popular
query : late and popular
query : lane and popular
query : lose and popular
query : lee and popular
[8, 7]
docs/Food Fight Erupted in Prison.txt
docs/A Festival of Books.txt


# TESTS

# Input – Information Retrieval System

In [786]:
test = searchEngine()

test.input(['document1.txt','document2.txt','document3.txt'])

document 1 : document1.txt
document 2 : document2.txt
document 3 : document3.txt


In [789]:
result = test.findWildCard("exa*le AND content")
print(result)
for file in [directory_input[index-1] for index in result]:
    print(file)

query : example and content
[2, 3]
docs/Rentals at the Oceanside Community.txt
docs/Gasoline Prices Hit Record High.txt


[]
